#### Task (20 Points)

In [ ]:
""" 
    topic: Moore-Bellman-Ford (MBF) Algorithm

    a)
    implement MBF()
        - digraph 
        - conservative edges
        - given a start node S
        - calculate the shortest path to all accesible nodes
        - test you function with a small example

    b)
    extend a) for non conservative
        - return a circle with negative weights

    c)
    extend the graph class, such that it allows weights
        - G = (V,E), c: E --> R_+
        - given the file waren.txt
            - the weight for an edge (A,B)∈E represents the trading rate from A to B
            - for one unit of commodity A, you therefore get c(a, b) units of commodity B
            - implement a algorithm that checks whether there is a cycle C of exchange transactions 
              in which you have more of a good after one cycle than before
                - r_C = \prod_(A,B)∈C c((A,B)) > 1
            - if there exists at least one Cycle C, then return the goods responsible for this
"""

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt

class Graph:
    invalid_node = -1
    def __init__(self, numorfile, directed=False):
        self._directed = directed
        self.conservative = False
        if type(numorfile) == int:
            self._nodes = [self.Node() for i in range(numorfile)]
        elif type(numorfile) == str:
            with open(numorfile, "r") as file:
                numnodes = int(file.readline())
                weight = [float(line.strip().split()[-1]) for line in file.readlines()]
                self._nodes = [self.Node(weight[i]) for i in range(numnodes)]
                for line in file:
                    tail = int(line.split()[0])
                    head = int(line.split()[1])
                    if tail != head:
                        self.add_edge(tail, head)
                    else:
                        raise RuntimeError("Invalid file format: loops not allowed")
                
        else:
            raise NotImplementedError("Type of Argument not allowed")

    def num_nodes(self):
        return len(self._nodes)

    def add_nodes(self, num_new_nodes):
        self._nodes.extend([self.Node() for i in range(num_new_nodes)])

    def add_edge(self, tail, head):
        if tail >= self.num_nodes() or tail < 0 or head >= self.num_nodes() or head < 0:
            raise ValueError("Edge cannot be added due to undefined endpoint")
        self._nodes[tail].add_neighbor(head)
        if not self._directed:
            self._nodes[head].add_neighbor(tail)

    def get_node(self, nodeId):
        if nodeId < 0 or nodeId >= self.num_nodes():
            raise ValueError("Invalid nodeId")
        return self._nodes[nodeId]

    def __str__(self):  # for printing
        out = ""
        if self._directed:
            out += "Digraph "
        else:
            out += "Undirected Graph "
        out += "with {} vertices, numbered 0, ..., {}\n".format(
            self.num_nodes(), self.num_nodes() - 1
        )
        for nodeId in range(self.num_nodes()):
            if self._directed:
                s = "leaving"
            else:
                s = "incident to"
            out += "The following edges are " + s + " vertex {}:\n".format(nodeId)
            for neighbor in self._nodes[nodeId].adjacent_nodes():
                out += "{} - {}\n".format(nodeId, neighbor.id())
        return out

    def is_simple(self):
        for nodeId in range(self.num_nodes()):
            nachbarsliste = []
            for neighbor in self._nodes[nodeId].adjacent_nodes():
                nachbarsliste.append(neighbor.id())

            unique_nachbarsliste = []
            for nachbar in nachbarsliste:
                if nachbar not in unique_nachbarsliste:
                    unique_nachbarsliste.append(nachbar)
                else:
                    return False

        return True

    def adjancency_matrix(self):
        num_nodes = self.num_nodes()
        matrix = [[0] * num_nodes for _ in range(num_nodes)]

        for nodeId in range(num_nodes):
            neighbors = self._nodes[nodeId].adjacent_nodes()
            for neighbor in neighbors:
                matrix[nodeId][neighbor.id()] = 1
        return np.array(matrix)
    
    def plot(self):
        if len(self._nodes) == 0:
            print("Graph is empty.")
            return

        for nodeId in range(self.num_nodes()):
            x, y = self._nodes[nodeId].coordinates()

            for neighbor in self._nodes[nodeId].adjacent_nodes():
                neighbor_id = neighbor.id()
                nx, ny = self._nodes[neighbor_id].coordinates()
                plt.plot([x, nx], [y, ny], color='black',linewidth=0.4)
    
    def plot_graph(self):
        self.plot()
        plt.title("Graph Visualization")
        plt.show()

    def norm(self,x_s,y_s,x_t,y_t):
        x = pow(x_t - x_s,2)
        y = pow(y_t - y_s,2)
        return sqrt(x+y)

    def dijkstra(self, start_node, end_node):
        num_nodes = self.num_nodes()
        distance = [float('inf')] * num_nodes
        distance[start_node] = 0
        predecessor = [-1] * num_nodes
        visited = [False] * num_nodes

        for _ in range(num_nodes):
            min_distance = float('inf')
            min_node = -1
            for node_id in range(num_nodes):
                if not visited[node_id] and distance[node_id] < min_distance:
                    min_distance = distance[node_id]
                    min_node = node_id
            visited[min_node] = True

            for neighbor in self._nodes[min_node].adjacent_nodes():
                neighbor_id = neighbor.id()
                edge_weight = self.norm(*self._nodes[min_node].coordinates(), *self._nodes[neighbor_id].coordinates())

                if not visited[neighbor_id] and distance[min_node] + edge_weight < distance[neighbor_id]:
                    distance[neighbor_id] = distance[min_node] + edge_weight
                    predecessor[neighbor_id] = min_node

        path = []
        current_node = end_node
        while current_node != -1:
            path.insert(0, current_node)
            current_node = predecessor[current_node]

        return path

    def plot_path(self,start,end):
        path = self.dijkstra(start,end)
        self.plot()
        for i in range(len(path) - 1):
            x1, y1 = self._nodes[path[i]].coordinates()
            x2, y2 = self._nodes[path[i + 1]].coordinates()
            plt.plot([x1, x2], [y1, y2], color='red', linewidth=2)

        plt.title("Dijkstra's Shortest Path")
        plt.show()

    class Node:
        def __init__(self,weight=None):
            self._neighbors = []
            self.weight = weight

        def add_neighbor(self, nodeId):
            self._neighbors.append(Graph.Neighbor(nodeId))

        def adjacent_nodes(self):
            return self._neighbors

        def number_of_neighbors(self):
            return len(self._neighbors)
        
        def weight(self):
            return self.weight

    class Neighbor:
        def __init__(self, nodeId):
            self._id = nodeId

        def id(self):
            return self._id

In [27]:
g = Graph("waren.txt",directed=True)

[0.06640123778891835, 2.1252703851088612, 0.00216465240446269, 0.14922648716804093, 0.7537889729335066, 0.2931036967068511, 0.6046436468704, 5.092244964589966, 0.00757980140485778, 0.0052695082973889884, 0.045800498001290366, 31.378529452577435, 15.648924268225803, 10.784111840437781, 21.896650901459356, 8285.788333098935, 119.3362364830002, 1043.172531224787, 32.51211180208039, 13.536921376428147, 40.17814605211459, 6.733004748179319, 203.59986598551282, 105.7263868833422, 4826.375984104626, 1.357822104889161, 43.14318954412392, 1.1791989652501522, 2114.3383115247498, 7.10306756634146, 36.38902605086291, 613.4274650405912, 5820.543269321986, 118.32855144652008, 1.2505730229899374, 46.434166481512754, 27.723294959155172, 3.259000366176434, 56.78970993204396, 552.1447537535728, 9.691874298096161, 297.3616916270155, 79.17744394460348, 78.18416971528872, 31.78517425531374, 281.52190977457354, 180.4932374005084, 3260.1016100640813, 3085.6338283541854, 64.25947870430599, 1.9826028035207106,

In [28]:
g.get_node(0).weight

0.06640123778891835